In [ ]:
import datasets

In [ ]:
dataset1 = datasets.load_dataset('HAERAE-HUB/KOREAN-WEBTEXT', split='train')
dataset1

In [ ]:
dataset2 = datasets.load_dataset('blueapple8259/c4-ko-cleaned-2', split='train')
dataset2

In [ ]:
dataset3 = datasets.load_dataset('HAERAE-HUB/KOREAN-SyntheticText-1.5B', split='train')
dataset3

In [ ]:
# maywell/korean_textbooks 데이터셋에서 분류기로 3점 이상의 데이터만 수집
dataset4 = datasets.load_dataset("devngho/korean-textbooks-edu", name="scored_over_3", split="train")
dataset4

In [ ]:
# 청와대 국민청원
dataset5 = datasets.load_dataset("heegyu/korean-petitions", split="train")
# dataset5에서 content 필드명을 text로 변경
dataset5 = dataset5.rename_column("content", "text")
dataset5

In [ ]:
from datasets import concatenate_datasets

combined_dataset = concatenate_datasets([dataset1, dataset2, dataset3, dataset4, dataset5])
combined_dataset

In [ ]:
import os
import re
from bs4 import BeautifulSoup
import hanja
from bs4 import MarkupResemblesLocatorWarning, XMLParsedAsHTMLWarning
import warnings

# =============================================================================
# PART 1: 모든 전처리 함수 정의
# =============================================================================

def clean_text(text):
    """
    논의된 모든 텍스트 정제 및 정규화 규칙을 순서대로 적용하는 함수
    """

    if not isinstance(text, str):
        return "" # 문자열이 아니면 빈 문자열 반환하거나 오류 처리

    # 0-1. UTF-8 유효성을 강제로 확인 및 유효하지 않은 문자 제거 (추가된 부분)
    # 이 과정에서 유효하지 않은 UTF-8 바이트 시퀀스가 제거됩니다.
    # 즉, 파이썬 문자열 내부에서 UTF-8로 다시 인코딩될 수 없는 문자를 제거합니다.
    try:
        text = text.encode('utf-8', errors='ignore').decode('utf-8')
    except Exception as e:
        # 이 예외는 이론적으로 발생하지 않아야 하지만, 만약을 위해 로깅합니다.
        print(f"Warning: Error during UTF-8 re-encoding/decoding: {e}. Original text: {text[:50]}...")
        text = "" # 오류 발생 시 해당 텍스트를 비움


    # 1-1. HTML 태그 제거
    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)
    text = BeautifulSoup(text, "html.parser").get_text()
    
    # 1-2. URL 및 이메일 주소 제거
    url_pattern = r'https?://[^\sㄱ-힣]+|www\.[^\sㄱ-힣]+'
    email_pattern = r'[^\s@]+@[^\sㄱ-힣]+' 
    text = re.sub(url_pattern, '[url placeholder]', text)
    text = re.sub(email_pattern, '[email protected]', text)

    # 1-3. 한자를 한글로 변환
    text = hanja.translate(text, 'substitution')
    # 이후 단어(단어), 단어 (단어) 와 같이 단어 뒤에 바로 같은 단어가 괄호로 감싸져 오는 경우, 뒤에 (단어) 를 제거하도록 필터 추가
    text = re.sub(r'(\S+)\s+\(\1\)', r'\1', text)

    # # 1-4. 불필요한 특수문자 제거
    # text = re.sub(r'[^\s\x20-\x7Eㄱ-ㅣ가-힣]', '', text)

    
    # 1-5. 반복 문자 처리 (ㅋㅋ, ㅎㅎ 등)
    text = re.sub(r'([ㄱ-ㅎㅏ-ㅣ])\1{2,}', r'\1\1', text)
    
    # 1-6. fix UnicodeEncodeError: 'utf-8' codec can't encode character '\udd2b' in position 2095: surrogates not allowed
    text = re.sub(r'[\uD800-\uDFFF]', '', text)

    return text

# 중복 제거를 위한 전역 세트(set) 선언
seen_texts = set()

def is_high_quality_and_unique(example):
    """
    품질 필터링(길이)과 중복 제거를 동시에 수행하는 함수
    """
    text = example['text']
    
    # 2-1. 길이 필터링: 텍스트 길이가 100글자 미만이면 탈락
    if len(text) < 100:
        return False
    
    # 2-2. 중복 필터링: 이미 등장한 텍스트면 탈락
    if text in seen_texts:
        return False
    
    # 모든 필터를 통과한 경우, seen_texts에 추가하고 통과 처리
    seen_texts.add(text)
    return True


max_num_proc= int(os.cpu_count() / 3)


print("\n2. 텍스트 정제 및 정규화를 시작합니다... (.map)")
cleaned_dataset = combined_dataset.map(
    lambda example: {'text': clean_text(example['text'])},
    num_proc=max_num_proc,
)
print("✅ 텍스트 정제 완료")
print("정제 후 데이터셋 정보:", cleaned_dataset)

# --- 3. 품질 및 중복 필터링 (.filter) ---
# 정제된 텍스트를 기준으로 길이 필터링 및 중복 제거를 수행합니다.
print("\n3. 품질 및 중복 필터링을 시작합니다... (.filter)")
final_dataset = cleaned_dataset.filter(
    is_high_quality_and_unique,
    num_proc=1 # 'seen_texts' 세트는 전역 변수이므로 다중 처리(num_proc > 1) 시 충돌할 수 있습니다.
                # 대용량 데이터 처리 시에는 다른 중복 제거 방식이 필요할 수 있습니다.
)
print("✅ 필터링 완료")
print("\n--- 최종 결과 ---")
print("최종 데이터셋 정보:", final_dataset)

In [ ]:
# text 컬럼만 남기고 나머지 컬럼 제거
columns_to_remove = [col for col in cleaned_dataset.column_names if col != "text"]
text_only_dataset = cleaned_dataset.remove_columns(columns_to_remove)
text_only_dataset

In [ ]:
shuffled_dataset = text_only_dataset.shuffle(seed=5768112)

In [ ]:
shuffled_dataset.push_to_hub("minpeter/tiny-ko-corpus", split="train")